In [1]:
import pandas

#### Use Pandas read_csv to load the us-counties.csv file.

The columns of the CSV file are:

date   - The date as a string (don't forget to convert this to a `pd.datetime`)

county - The name of the county

state  - The name of the state

fips   - The fips code of the county (the first two digits represent the state, the last three, the county)

cases  - The cumulative number of cases at the date in the county

deaths - The cumulative number of deaths in at the date in the county

#### Question 1: Produce a DataFrame that contains only the data for the top 10 Counties by total number of cases

####  Question 2: Produce a DataFrame that contains only the data for the top 5 states by total number of cases

#### Question 3: Calculate the incidence (the number of new cases per day... i.e. the difference between the cases on a day - the cases on a previous day) for the top 10 Counties.

#### Question 4: Calculate the peak incidence for the 10 counties (the day with the highest number of new cases).

#### Question 5: Plot an incidence curve for the County with the highest peak.

#### Question 6: Plot a vertical bar graph with County as the category and the peak incidence as the measure.